# ETL (Extrair, Transformar e Carregar) da camada Raw para Silver
Este notebook tem como objetivo aplicar, em nível moderado, a transformação e limpeza dos dados brutos da camada Raw para a camada Silver, a fim de fornecer uma “visão corporativa” da principal entidade do negócio. Ele processa os dados brutos do banco de dados da Comunicação de Acidente de Trabalho (CAT) do INSS.

# Configuração
Configuração de ambiente, necessária para o funcionamento pleno do ETL.

### Instalar e Importar



In [ ]:
!pip install pyspark
!pip install psycopg2

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, when, avg, from_unixtime, unix_timestamp, lit, regexp_replace, to_date
from pyspark.sql.types import StringType

from psycopg2 import extras

import re
import psycopg2

### Iniciar seção no Spark

In [ ]:
spark = SparkSession.builder \
    .appName("CSVtoPostgresETL") \
    .getOrCreate()

### Variáveis

In [ ]:
limite_repeticao = 0.95
limite_nulos = 0.60

colunas_a_remover = []
novas_colunas = []

colunas_limpas = set()
colunas_a_remover = []

caminho_arquivo = "../Data layer/raw/dados_brutos.csv"

### Funções Auxiliares

In [ ]:
def tirar_numero(coluna):
  return re.sub(r'\d+', '', coluna)

def remover_codigo(coluna):
    return regexp_replace(col(coluna), r"^[A-Z0-9.]+[-\s]+", "")

# Extração
Seção de extração dos dados brutos do arquivo CSV



In [ ]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../Data layer/raw/dados_brutos.csv")

df.show()
df.printSchema()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/14 21:32:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+--------------------------+--------------+--------------------+--------------------+------------------+----------------------------+-------------------+--------------------+-----------------+---------------------+--------------------+--------------------+---------------------------+--------------------+---------+----------------+--------------------+--------------------+---------------+-----------------------+---------------+---------------+----------------+-------------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE2.0 Empregador|CNAE2.0 Empregador Descrição|       Emitente CAT|Espécie do benefício|Filiação Segurado|Indica Óbito Acidente|          Munic Empr|   Natureza da Lesão|Origem de Cadastramento CAT|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF  Munic.  Acidente|UF Munic. Empregador|Data Acidente18|Data Despacho Benefício|Data Acidente20|Data Nascimento|Data Emissão CAT|CNPJ/CEI Empregador|
+--------------------------+----

26/01/14 21:32:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Agente  Causador  Acidente, Data Acidente, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Emitente CAT, Espécie do benefício, Filiação Segurado, Indica Óbito Acidente, Munic Empr, Natureza da Lesão, Origem de Cadastramento CAT, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Acidente, Data Despacho Benefício, Data Acidente, Data Nascimento, Data Emissão CAT, CNPJ/CEI Empregador
 Schema: Agente  Causador  Acidente, Data Acidente1, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Emitente CAT, Espécie do benefício, Filiação Segurado, Indica Óbito Acidente, Munic Empr, Natureza da Lesão, Origem de Cadastramento CAT, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Acidente18, Data Despacho Benefício, Data Acidente20, Data Nascimento, Data Emissão CAT, CNPJ/CEI Empregador
Expec

# Transformação
Nesta seção, iremos transformar e limpar os dados vindos da camada RAW para a camada Silver, ainda utilizando o Spark.

### Substituição de valores não classificados por valor nulo

In [ ]:
nao_classif_string = '{ñ class}'
nao_classif_numeric = 0
nao_classif_date = '00/00/0000'

total_linhas = df.count()

for coluna in df.columns:
    tipo_coluna = df.schema[coluna].dataType
    coluna_escaped = col(f"`{coluna}`")

    if coluna in ["Data Nascimento", "Data Emissão CAT"]:
        df_temp = df.withColumn(coluna, when(coluna_escaped == nao_classif_date, None).otherwise(coluna_escaped))

        media_ts = df_temp.select(avg(unix_timestamp(coluna, "dd/MM/yyyy"))).collect()[0][0]

        if media_ts:
            data_media_str = spark.range(1).select(from_unixtime(lit(media_ts), "dd/MM/yyyy")).collect()[0][0]
            expressao = when((coluna_escaped == nao_classif_date) | (coluna_escaped.isNull()), data_media_str) \
                        .otherwise(coluna_escaped).alias(coluna)
        else:
            expressao = coluna_escaped.alias(coluna)

    elif isinstance(tipo_coluna, StringType):
        expressao = when(coluna_escaped == nao_classif_string, None).otherwise(coluna_escaped).alias(coluna)
    else:
        expressao = when(coluna_escaped == nao_classif_numeric, None).otherwise(coluna_escaped).alias(coluna)

    novas_colunas.append(expressao)

df_limpo = df.select(*novas_colunas)
df_limpo.show()

+--------------------------+--------------+--------------------+--------------------+------------------+----------------------------+-------------------+--------------------+-----------------+---------------------+--------------------+--------------------+---------------------------+--------------------+---------+----------------+--------------------+--------------------+---------------+-----------------------+---------------+---------------+----------------+-------------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE2.0 Empregador|CNAE2.0 Empregador Descrição|       Emitente CAT|Espécie do benefício|Filiação Segurado|Indica Óbito Acidente|          Munic Empr|   Natureza da Lesão|Origem de Cadastramento CAT|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF  Munic.  Acidente|UF Munic. Empregador|Data Acidente18|Data Despacho Benefício|Data Acidente20|Data Nascimento|Data Emissão CAT|CNPJ/CEI Empregador|
+--------------------------+----

26/01/14 21:32:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Agente  Causador  Acidente, Data Acidente, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Emitente CAT, Espécie do benefício, Filiação Segurado, Indica Óbito Acidente, Munic Empr, Natureza da Lesão, Origem de Cadastramento CAT, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Acidente, Data Despacho Benefício, Data Acidente, Data Nascimento, Data Emissão CAT, CNPJ/CEI Empregador
 Schema: Agente  Causador  Acidente, Data Acidente1, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Emitente CAT, Espécie do benefício, Filiação Segurado, Indica Óbito Acidente, Munic Empr, Natureza da Lesão, Origem de Cadastramento CAT, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Acidente18, Data Despacho Benefício, Data Acidente20, Data Nascimento, Data Emissão CAT, CNPJ/CEI Empregador
Expec

### Remoção de Colunas com Dados Não Classificados ou Constante
As características utilizadas para as colunas caírem nesse termo são:

- 95%+ dos valores são iguais
- 60%+ dos valores são nulos/não classificados
- Variância = 0
- Não agrega informação

In [ ]:
for coluna in df_limpo.columns:
    df_limpo = df_limpo.withColumnRenamed(coluna, coluna.replace(".", ""))    

for coluna in df_limpo.columns:
    # 1. Verifica Porcentagem de Nulos
    # Como já retiramos os não classificados, agora só contar os nulos diretamente
    qtd_nulos = df_limpo.select(count(when(col(coluna).isNull(), 1))).collect()[0][0]

    porcentagem_nulos = (qtd_nulos / total_linhas) * 100

    if porcentagem_nulos > (limite_nulos * 100):
        print(f"Removendo {coluna}: {porcentagem_nulos:.2f}% de nulos (incluindo não classificados convertidos).")
        colunas_a_remover.append(coluna)
        continue

    # 2. Verifica valores repetitivos (>95%)
    repeticao_maxima = df_limpo.groupBy(coluna).count() \
        .agg({"count": "max"}).collect()[0][0]

    if repeticao_maxima is None: repeticao_maxima = 0

    porcentagem_maxima = (repeticao_maxima / total_linhas) * 100
    print(f"{coluna}: {porcentagem_maxima:.2f}% de valor repetido.")

    if porcentagem_maxima > (limite_repeticao * 100):
        print(f"Removendo {coluna}: {porcentagem_maxima:.2f}% de valor repetido.")
        colunas_a_remover.append(coluna)

if colunas_a_remover:
    df_final = df_limpo.drop(*colunas_a_remover)
    print(f"\nTotal removido: {len(colunas_a_remover)}")
else:
    df_final = df_limpo
    print("\nNenhuma coluna removida.")

df_final.show()

26/01/14 21:32:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Acidente
 Schema: Data Acidente1
Expected: Data Acidente1 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


Agente  Causador  Acidente: 6.00% de valor repetido.


26/01/14 21:32:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Acidente
 Schema: Data Acidente1
Expected: Data Acidente1 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


Data Acidente1: 33.08% de valor repetido.
CBO: 10.91% de valor repetido.
CID-10: 4.99% de valor repetido.
CNAE20 Empregador: 9.60% de valor repetido.
CNAE20 Empregador Descrição: 9.60% de valor repetido.
Emitente CAT: 95.82% de valor repetido.
Removendo Emitente CAT: 95.82% de valor repetido.
Espécie do benefício: 100.00% de valor repetido.
Removendo Espécie do benefício: 100.00% de valor repetido.
Filiação Segurado: 99.13% de valor repetido.
Removendo Filiação Segurado: 99.13% de valor repetido.
Indica Óbito Acidente: 99.54% de valor repetido.
Removendo Indica Óbito Acidente: 99.54% de valor repetido.
Munic Empr: 9.06% de valor repetido.
Natureza da Lesão: 17.60% de valor repetido.
Origem de Cadastramento CAT: 100.00% de valor repetido.
Removendo Origem de Cadastramento CAT: 100.00% de valor repetido.
Parte Corpo Atingida: 22.24% de valor repetido.
Sexo: 64.23% de valor repetido.
Tipo do Acidente: 74.23% de valor repetido.
UF  Munic  Acidente: 34.19% de valor repetido.
UF Munic Empreg

26/01/14 21:33:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Acidente
 Schema: Data Acidente18
Expected: Data Acidente18 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv
26/01/14 21:33:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Acidente
 Schema: Data Acidente18
Expected: Data Acidente18 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


Data Acidente18: 33.08% de valor repetido.
Data Despacho Benefício: 100.00% de valor repetido.
Removendo Data Despacho Benefício: 100.00% de valor repetido.


26/01/14 21:33:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Acidente
 Schema: Data Acidente20
Expected: Data Acidente20 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv
26/01/14 21:33:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Acidente
 Schema: Data Acidente20
Expected: Data Acidente20 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


Data Acidente20: 1.50% de valor repetido.
Data Nascimento: 0.73% de valor repetido.
Data Emissão CAT: 1.80% de valor repetido.
CNPJ/CEI Empregador: 96.25% de valor repetido.
Removendo CNPJ/CEI Empregador: 96.25% de valor repetido.

Total removido: 7
+--------------------------+--------------+--------------------+--------------------+-----------------+---------------------------+--------------------+--------------------+--------------------+---------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE20 Empregador|CNAE20 Empregador Descrição|          Munic Empr|   Natureza da Lesão|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF  Munic  Acidente|UF Munic Empregador|Data Acidente18|Data Acidente20|Data Nascimento|Data Emissão CAT|
+--------------------------+--------------+--------------------+--------------------+-----------

26/01/14 21:33:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Agente  Causador  Acidente, Data Acidente, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Acidente, Data Acidente, Data Nascimento, Data Emissão CAT
 Schema: Agente  Causador  Acidente, Data Acidente1, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Acidente18, Data Acidente20, Data Nascimento, Data Emissão CAT
Expected: Data Acidente1 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


### Remoção de Colunas Altamente Correlacionada
Serão identificadas e removidas colunas que possuem conteúdo idêntico ou derivado direto de outras colunas existentes, eliminando redundâncias.

In [ ]:
for coluna in df.columns:
  nome_limpo = tirar_numero(coluna).lower()
  print(nome_limpo)

  if nome_limpo in colunas_limpas:
    print("Removendo coluna: ", nome_limpo)
    colunas_a_remover.append(coluna)
  else:
    colunas_limpas.add(nome_limpo)


df_final = df_final.drop(*colunas_a_remover)
df_final.show()

agente  causador  acidente
data acidente
cbo
cid-
cnae. empregador
cnae. empregador descrição
emitente cat
espécie do benefício
filiação segurado
indica óbito acidente
munic empr
natureza da lesão
origem de cadastramento cat
parte corpo atingida
sexo
tipo do acidente
uf  munic.  acidente
uf munic. empregador
data acidente
Removendo coluna:  data acidente
data despacho benefício
data acidente
Removendo coluna:  data acidente
data nascimento
data emissão cat
cnpj/cei empregador
+--------------------------+--------------+--------------------+--------------------+-----------------+---------------------------+--------------------+--------------------+--------------------+---------+----------------+-------------------+-------------------+---------------+----------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE20 Empregador|CNAE20 Empregador Descrição|          Munic Empr|   Natureza da Lesão|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF 

26/01/14 21:33:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Agente  Causador  Acidente, Data Acidente, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Nascimento, Data Emissão CAT
 Schema: Agente  Causador  Acidente, Data Acidente1, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Nascimento, Data Emissão CAT
Expected: Data Acidente1 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


## Tratando/Limpando valores nulos

O tratamento dos campos nulos foi definido com base no impacto analítico de cada variável:

(Agente Causador, CBO e UF. Munic. do Acidente): Valores nulos serão alterados para o termo "Não identificado". Isso preserva o registro para contagem total, já que outras variáveis permitem análises parciais;

(CID e CNAE): São campos essenciais. Sem o CID, perde-se a causa médica; sem o CNAE, inviabiliza-se a análise por setor econômico. Se nulos, serão descartados, pois a falta desses dados inviabiliza a análise.


In [6]:
cols_para_preencher = ["Agente  Causador  Acidente", "CBO", "`UF  Munic  Acidente`"]
cols_criticas_para_drop = ["CID-10", "`CNAE20 Empregador`", "Data Nascimento"]

df_tratado = df_final \
    .na.fill("Não identificado", subset=cols_para_preencher) \
    .na.drop(subset=cols_criticas_para_drop)

print(f"Total antes: {df_final.count()}")
print(f"Total depois: {df_tratado.count()}")

df_tratado.show()

Total antes: 157845
Total depois: 145888
+--------------------------+--------------+--------------------+--------------------+-----------------+---------------------------+--------------------+--------------------+--------------------+---------+----------------+-------------------+-------------------+---------------+----------------+
|Agente  Causador  Acidente|Data Acidente1|                 CBO|              CID-10|CNAE20 Empregador|CNAE20 Empregador Descrição|          Munic Empr|   Natureza da Lesão|Parte Corpo Atingida|     Sexo|Tipo do Acidente|UF  Munic  Acidente|UF Munic Empregador|Data Nascimento|Data Emissão CAT|
+--------------------------+--------------+--------------------+--------------------+-----------------+---------------------------+--------------------+--------------------+--------------------+---------+----------------+-------------------+-------------------+---------------+----------------+
|      Rua e Estrada - S...|       2023/03|252105-Administrador|S42.0 Frat

26/01/14 21:33:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Agente  Causador  Acidente, Data Acidente, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Nascimento, Data Emissão CAT
 Schema: Agente  Causador  Acidente, Data Acidente1, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Nascimento, Data Emissão CAT
Expected: Data Acidente1 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv


## Retirando prefixos dos dados
Retirando prefixos de números e códigos dos dados, como '1234-<dado>' e 'S00.0 <dado>'.

In [ ]:
df_limpo = (
    df_tratado
        .withColumn("CID-10", remover_codigo("CID-10"))
        .withColumn("CBO", remover_codigo("CBO"))
        .withColumn("Munic Empr", remover_codigo("Munic Empr"))
)

df_limpo.tail(2)

[Row(Agente  Causador  Acidente='Impacto de Pes. Contra Objeto Parado', Data Acidente1='2023/02', CBO='Servente de Obras', CID-10='Frat da Extremidade Distal do Radio', CNAE20 Empregador=4313, CNAE20 Empregador Descrição='Obras de Terraplenagem', Munic Empr='Ignorado', Natureza da Lesão='Escoriacao, Abrasao (Ferimento Superficial)', Parte Corpo Atingida='Joelho', Sexo='Masculino', Tipo do Acidente='Trajeto', UF  Munic  Acidente='Zerado', UF Munic Empregador='Zerado', Data Nascimento='04/10/1999', Data Emissão CAT='01/03/2023'),
 Row(Agente  Causador  Acidente='Veiculo, Nic', Data Acidente1='2023/01', CBO='Emendador de Cabos Elétricos e Telefôn', CID-10='Outr Traum Espec Envolv Regioes Mult Co', CNAE20 Empregador=4221, CNAE20 Empregador Descrição='Obras para Geracao e Distribuicao de Energia', Munic Empr='Bauru', Natureza da Lesão='Lesao Imediata', Parte Corpo Atingida='Partes Multiplas - Aplica-Se Quando Mais de U', Sexo='Masculino', Tipo do Acidente='Trajeto', UF  Munic  Acidente='Mar

# Carregamento
Seção final do processo ETL, os dados são persistidos em um destino final, neste caso, um banco de dados PostgreSQL.

### Configuração

In [ ]:
# Mapeamento de nomes (De -> Para)
mapeamento_colunas = {
    "Agente  Causador  Acidente": "agente_causador_acidente",
    "Data Acidente1": "data_acidente_referencia",
    "CBO": "cbo_codigo_descricao",
    "CID-10": "cid_10",
    "CNAE20 Empregador": "cnae_empregador",
    "CNAE20 Empregador Descrição": "cnae_empregador_descricao",
    "Natureza da Lesão": "natureza_lesao",
    "Parte Corpo Atingida": "parte_corpo_atingida",
    "Sexo": "sexo",
    "Tipo do Acidente": "tipo_acidente",
    "Munic Empr": "municipio_empregador",
    "UF  Munic  Acidente": "uf_municipio_acidente",
    "UF Munic Empregador": "uf_municipio_empregador",
    "Data Nascimento": "data_nascimento",
    "Data Emissão CAT": "data_emissao_cat"
}

# Aplicando a renomeação e casting de data
df_load = df_limpo
for col_origem, col_destino in mapeamento_colunas.items():
    df_load = df_load.withColumnRenamed(col_origem, col_destino)

df_load = df_load \
    .withColumn("data_nascimento", to_date("data_nascimento", "dd/MM/yyyy")) \
    .withColumn("data_emissao_cat", to_date("data_emissao_cat", "dd/MM/yyyy")) \
    .withColumn("data_acidente_referencia", to_date("data_acidente_referencia", "dd/MM/yyyy")) 
        
    

# Preparação dos dados para o Driver
dados_para_inserir = [tuple(x) for x in df_load.collect()]
colunas = df_load.columns

# Configuração da conexão com Psycopg2
conn_params = {
    "host": "localhost",
    "database": "cat_db",
    "user": "admin",
    "password": "admin",
    "port": 5432
}

26/01/14 21:33:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Agente  Causador  Acidente, Data Acidente, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Nascimento, Data Emissão CAT
 Schema: Agente  Causador  Acidente, Data Acidente1, CBO, CID-10, CNAE2.0 Empregador, CNAE2.0 Empregador Descrição, Munic Empr, Natureza da Lesão, Parte Corpo Atingida, Sexo, Tipo do Acidente, UF  Munic.  Acidente, UF Munic. Empregador, Data Nascimento, Data Emissão CAT
Expected: Data Acidente1 but found: Data Acidente
CSV file: file:///home/yabamiah/Sandbox/cat-analytics/Data%20layer/raw/dados_brutos.csv
                                                                                

Carga de 145888 registros concluída com sucesso!


### Load
Realiza a carga dos dados transformados para o banco de dados PostgreSQL.

In [ ]:
try:
    # Estabelece a conexão
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    # Execução do Insert usando execute_values
    query = f"INSERT INTO ACIDENTE ({', '.join(colunas)}) VALUES %s"

    extras.execute_values(
        cursor,
        query,
        dados_para_inserir,
        template=None,
        page_size=1000
    )

    conn.commit()
    print(f"Carga de {len(dados_para_inserir)} registros concluída com sucesso!")

except Exception as e:
    print(f"Erro ao inserir dados: {e}")
    if conn:
        conn.rollback()
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()